In [ ]:
from fastai2.vision.all import *
from fastai2.vision.widgets import *

In [ ]:
path = Path("models")

In [ ]:
learn_inf = load_learner(path/'tennis_with_clean_01.pkl', cpu=True)

In [ ]:
btn_upload = widgets.FileUpload()
out_pl = widgets.Output()
lbl_pred = widgets.Label()
btn_run = widgets.Button(description='Classify')

In [ ]:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:0.04f}'

In [ ]:
btn_run.on_click(on_click_classify)

In [ ]:
VBox([widgets.Label('Select your Tennis Shot!'), 
      btn_upload, btn_run, out_pl, lbl_pred])